# Import dependencies

In [ ]:
# uncomment and run the line below if you need to install the packages
# pip install geopandas pandas matplotlib shapely json os numpy

In [3]:
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Point
import json
import os

# Defining Functions

In [11]:
def convert_to_geodataframe_points(df):
    # Create a copy of the dataframe to avoid modifying the original
    gdf = df.copy()
    
    # Convert the coordinates to shapely Point objects
    gdf['geometry'] = gdf['coords'].apply(lambda x: Point(x[0], x[1]))  # Note: Point(lat, lon)
    
    # Create the GeoDataFrame
    gdf = gpd.GeoDataFrame(gdf, geometry='geometry')
    
    # Set the coordinate reference system (CRS) to WGS84
    gdf.set_crs(epsg=4326, inplace=True)
    # drop coords column
    gdf = gdf.drop(columns=['coords'])
    #rename index column
    gdf.rename(columns={"index": "bridge_index"}, inplace=True)
    
    return gdf

In [12]:
def drop_unused_columns(df):
    # drop columns that are not needed
    drop_cols = ["type", "geometry", "subregion_indices"]
    for c in df.columns:
        if c.startswith("used_by_cells"):
            drop_cols.append(c)
    for c in df.columns:
        if "_fixed" in c:
            drop_cols.append(c)
    return df.drop(columns=drop_cols)

In [13]:
def convert_array_columns(df):
    df_fixed = df.copy()
    print(f"Starting conversion process for {len(df.columns)} columns")

    # Loop through each column
    complex_cols_found = 0
    for i, col in enumerate(df.columns):
        print(f"Processing column {i+1}/{len(df.columns)}: '{col}'")
        
        # Check if column contains complex data types
        has_complex = df[col].apply(lambda x: isinstance(x, (np.ndarray, list, dict))).any()
        
        if has_complex:
            complex_cols_found += 1
            print(f"  Found complex data in column '{col}' - converting...")
            
            # Sample the first complex value for debugging
            sample_idx = df[col].apply(lambda x: isinstance(x, (np.ndarray, list, dict))).idxmax()
            sample_val = df.loc[sample_idx, col]
            print(f"  Sample value before conversion: {type(sample_val)}, first few items: {str(sample_val)[:50]}...")
            
            # Convert complex values to JSON strings
            df_fixed[col] = df[col].apply(
                lambda x: json.dumps(x.tolist() if isinstance(x, np.ndarray) else x) 
                if isinstance(x, (np.ndarray, list, dict)) else x
            )
            
            # Verify conversion worked
            new_sample_val = df_fixed.loc[sample_idx, col]
            print(f"  Sample value after conversion: {type(new_sample_val)}, preview: {str(new_sample_val)[:50]}...")

    print(f"Conversion complete! Found and converted {complex_cols_found} columns with complex data types")
    
    return df_fixed

In [14]:
def process_data(file_path):
    # Load the data
    df = pd.read_parquet(file_path)
    print(f"Loaded data with shape: {df.shape}")

     # Drop unused columns
    df = drop_unused_columns(df)
    print(f"Dropped unused columns, new shape: {df.shape}")

    # Convert the data to a GeoDataFrame
    df = convert_to_geodataframe_points(df)
    print(f"Converted to GeoDataFrame with shape: {df.shape}")

    # Convert array columns to JSON strings
    df = convert_array_columns(df)
    print(f"Converted columns. New shape: {df.shape}")

    return df

# File Paths

In [4]:
# paths
local_path = "/Volumes/samsung-4tb/b2p/impact-model/no_order_1_less_than_500m_with_top_sites/model_outputs/"
bridge_view_path = "joined_data/bridge_view_data.parquet"
output_folder_path ="/Volumes/samsung-4tb/b2p/impact-model/cleaned_data/"

et_path = os.path.join(local_path, "ethiopia/",bridge_view_path)
civ_path = os.path.join(local_path, "ivory_coast/",bridge_view_path)
kenya_path = os.path.join(local_path, "kenya/",bridge_view_path)
rwanda_path = os.path.join(local_path, "rwanda/",bridge_view_path)
tanzania_path = os.path.join(local_path, "tanzania/",bridge_view_path)
uganda_path = os.path.join(local_path, "uganda/",bridge_view_path)
zambia_path = os.path.join(local_path, "zambia/",bridge_view_path)


In [5]:
tiny_bridges_path = "/Volumes/samsung-4tb/b2p/impact-model/cleaned_data/all_bridges_tiny.geojson"
tiny_bridges_tiles_path = "/Volumes/samsung-4tb/b2p/impact-model/cleaned_data/all_bridges_tiny_tiles.mbtiles"
full_bridges_path = "/Volumes/samsung-4tb/b2p/impact-model/cleaned_data/all_bridges.geojson"
full_bridges_tiles_path = "/Volumes/samsung-4tb/b2p/impact-model/cleaned_data/full_bridges_tiles.mbtiles"

# Process data to export to geojson

In [ ]:
et_bridges = process_data(et_path)
civ_bridges = process_data(civ_path)
kenya_bridges = process_data(kenya_path)
rwanda_bridges = process_data(rwanda_path)
tanzania_bridges = process_data(tanzania_path)
uganda_bridges = process_data(uganda_path)
zambia_bridges = process_data(zambia_path)

In [ ]:
# join all geodataframes
all_bridges = pd.concat([et_bridges, civ_bridges, kenya_bridges, rwanda_bridges, tanzania_bridges, uganda_bridges, zambia_bridges])
all_bridges

In [ ]:
# write to geojson
all_bridges = convert_array_columns(all_bridges)
all_bridges = gpd.GeoDataFrame(all_bridges, geometry='geometry')
all_bridges.set_crs(epsg=4326, inplace=True)
all_bridges.to_file(os.path.join(output_folder_path, "all_bridges.geojson"), driver='GeoJSON')
print("Geojson file written successfully!")

## Creating Small File size Bridges Dataset with just bridge ID and geometry

In [ ]:
# run if all_bridges is not empty
# all_bridges = gpd.read_file(os.path.join(output_folder_path, "all_bridges.geojson"))

In [ ]:
# drop all columns except geometry and bridge_index
all_bridges_tiny = all_bridges[["bridge_index", "geometry"]]
all_bridges_tiny.to_file(os.path.join(output_folder_path, "all_bridges_tiny.geojson"), driver='GeoJSON')
print("Done!")

## Tileset Development

In [ ]:
print(tiny_bridges_path)
print (tiny_bridges_tiles_path)

In [ ]:
!tippecanoe -o {tiny_bridges_tiles_path} -r1 -zg {tiny_bridges_path} --force

In [ ]:
!tippecanoe -o {full_bridges_tiles_path} -pf -z12 -Z12 -r1 --extend-zooms-if-still-dropping {full_bridges_path} --force


# Process data for database

This is necessary because we want to array types to be conserved in the database, but geojson files cannot handle these types

In [ ]:
def read_in_as_parquet (file_path):
    # Load the data
    df = pd.read_parquet(file_path)
    print(f"Loaded {} data with shape: {df.shape}")

    # Convert the data to a GeoDataFrame
    df = convert_to_geodataframe_points(df)
    print(f"Converted to GeoDataFrame with shape: {df.shape}")

    return df

In [15]:
et_full = read_in_as_parquet(et_path)
civ_full = read_in_as_parquet(civ_path)
kenya_full = read_in_as_parquet(kenya_path)
rwanda_full = read_in_as_parquet(rwanda_path)
tanzania_full = read_in_as_parquet(tanzania_path)
uganda_full = read_in_as_parquet(uganda_path)
zambia_full = read_in_as_parquet(zambia_path)
print("all files read in as parquet")
# join all geodataframes
all_bridges_full = pd.concat([et_full, civ_full, kenya_full, rwanda_full, tanzania_full, uganda_full, zambia_full])

Loaded data with shape: (46246, 24)
Converted to GeoDataFrame with shape: (46246, 23)
Loaded data with shape: (9048, 24)
Converted to GeoDataFrame with shape: (9048, 23)
Loaded data with shape: (13191, 24)
Converted to GeoDataFrame with shape: (13191, 23)
Loaded data with shape: (1010, 24)
Converted to GeoDataFrame with shape: (1010, 23)
Loaded data with shape: (25616, 24)
Converted to GeoDataFrame with shape: (25616, 23)
Loaded data with shape: (5951, 24)
Converted to GeoDataFrame with shape: (5951, 23)
Loaded data with shape: (15212, 24)
Converted to GeoDataFrame with shape: (15212, 23)
all files read in as parquet


In [16]:
all_bridges_full

,bridge_index,type,geometry,subregion_indices,exit_point_index,used_by_cells_for_semi_dense_urban_optimal,used_by_h3_for_semi_dense_urban_optimal,used_by_cells_for_health_posts_optimal,used_by_h3_for_health_posts_optimal,used_by_cells_for_primary_schools_fixed,used_by_h3_for_primary_schools_fixed,used_by_cells_for_all_health_facilities_optimal,used_by_h3_for_all_health_facilities_optimal,used_by_cells_for_health_centers_optimal,used_by_h3_for_health_centers_optimal,used_by_cells_for_major_hospitals_optimal,used_by_h3_for_major_hospitals_optimal,used_by_cells_for_major_roads_optimal,used_by_h3_for_major_roads_optimal,used_by_cells_for_secondary_schools_fixed,used_by_h3_for_secondary_schools_fixed,used_by_cells_for_all_education_facilities_fixed,used_by_h3_for_all_education_facilities_fixed
0,302351,bridge_predicted,POINT (37.46229 4.84980),"[269620, 269629]",302352,"[[12062, 5346], [12063, 5345], [12064, 5344], ...","[886a4b769bfffff, 886a4b769bfffff, 886a4b7691f...","[[12062, 5346], [12063, 5345], [12064, 5344], ...","[886a4b769bfffff, 886a4b769bfffff, 886a4b7691f...",[],[],"[[12060, 5355], [12061, 5356], [12059, 5355], ...","[886a4b39a7fffff, 886a4b39a7fffff, 886a4b39a7f...","[[12062, 5346], [12063, 5345], [12064, 5344], ...","[886a4b769bfffff, 886a4b769bfffff, 886a4b7691f...",[],[],[],[],[],[],[],[]
1,302352,bridge_predicted,POINT (37.46595 4.85105),"[269620, 270228]",302353,"[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...","[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[],[],"[[12044, 5353], [12045, 5354], [12046, 5355], ...","[886a4b39a1fffff, 886a4b39a1fffff, 886a4b39a1f...","[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[],[],[],[],[],[],[],[]
2,302353,bridge_predicted,POINT (37.46508 4.85778),"[269906, 270228]",302354,"[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...","[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[],[],"[[12050, 5358], [12039, 5351], [12040, 5352], ...","[886a4b39a1fffff, 886a4b39a9fffff, 886a4b39a9f...","[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[],[],[],[],[],[],[],[]
3,302354,bridge_predicted,POINT (37.45727 4.86828),"[267568, 269906]",302355,"[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...","[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[],[],"[[12037, 5349], [12038, 5350]]","[886a4b39e7fffff, 886a4b39a9fffff]","[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[],[],"[[12037, 5349], [12038, 5350], [12039, 5351], ...","[886a4b39e7fffff, 886a4b39a9fffff, 886a4b39a9f...",[],[],[],[]
4,302355,bridge_predicted,POINT (37.45522 4.87006),"[267568, 269717]",302356,"[[12028, 5319], [12029, 5318], [12030, 5317], ...","[886a4b3937fffff, 886a4b3937fffff, 886a4b3937f...","[[12028, 5319], [12029, 5318], [12030, 5317], ...","[886a4b3937fffff, 886a4b3937fffff, 886a4b3937f...",[],[],"[[12032, 5326], [12033, 5325], [12034, 5324], ...","[886a4b76d9fffff, 886a4b76d9fffff, 886a4b76d9f...","[[12028, 5319], [12029, 5318], [12030, 5317], ...","[886a4b3937fffff, 886a4b3937fffff, 886a4b3937f...",[],[],"[[12034, 5330], [12035, 5330], [12035, 5339], ...","[886a4b76d9fffff, 886a4b76d9fffff, 886a4b76dbf...",[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15207,105971,bridge_predicted,POINT (29.63156 -8.49877),"[149739, 150797]",105972,"[[345, 9078], [347, 9079], [348, 9079], [349, ...","[8896a9a269fffff, 8896a9b197fffff, 8896a9b197f...","[[345, 9078], [347, 9079], [348, 9079], [349, ...","[8896a9a269fffff, 8896a9b197fffff, 8896a9b197f...","[[328, 9159], [329, 9158]]","[8896a9

In [18]:
# write to geoparquet file
all_bridges_full.to_parquet(os.path.join(output_folder_path, "all_bridges_full.parquet"), engine='pyarrow')

Upload Script

`ogr2ogr -f "PostgreSQL" PG:"host=fika-impact-database.cdoaycgiqb5w.us-west-1.rds.amazonaws.com dbname=postgres user=postgres password=PASSWORD_HERE" "all_bridges_full.parquet" -nln "bridges" -lco "GEOMETRY_NAME=geometry" --config PG_USE_COPY YES`